<a href="https://colab.research.google.com/github/hwarang97/paperswithcode/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive/')

!pip install torchinfo
from torchinfo import summary

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
cuda


In [ ]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1, bias=False, ),
            nn.BatchNorm2d(in_channels),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.block(x)
        return x

class Identity_Block(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block = nn.Sequential(
            BasicConv2d(in_channels),
            BasicConv2d(in_channels)
        )

    def forward(self, x):
        x = self.block(x)
        return x

class projection_Block(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, 2*in_channels, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(2*in_channels),
            nn.ReLU(),
            BasicConv2d(2*in_channels)
        )

    def forward(self, x):
        x = self.block(x)
        return x

class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_2 = nn.Sequential(
            Identity_Block(64),
            Identity_Block(64),
            Identity_Block(64)
        )
        self.conv_3 = nn.Sequential(
            projection_Block(64),
            Identity_Block(128),
            Identity_Block(128),
            Identity_Block(128)
        )
        self.conv_4 = nn.Sequential(
            projection_Block(128),
            Identity_Block(256),
            Identity_Block(256),
            Identity_Block(256)
        )
        self.conv_5 = nn.Sequential(
            projection_Block(256),
            Identity_Block(512),
            Identity_Block(512)
        )
        self.gap = nn.AvgPool2d(7)
        self.fc = nn.Linear(512,1000)

    def forward(self, x):
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.conv_5(x)
        x = self.gap(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

In [ ]:
model = ResNet()
model.train()

summary(model, input_size=(10,64,56,56))

Layer (type:depth-idx)                             Output Shape              Param #
ResNet                                             [10, 1000]                --
├─Sequential: 1-1                                  [10, 64, 56, 56]          --
│    └─Identity_Block: 2-1                         [10, 64, 56, 56]          --
│    │    └─Sequential: 3-1                        [10, 64, 56, 56]          73,984
│    └─Identity_Block: 2-2                         [10, 64, 56, 56]          --
│    │    └─Sequential: 3-2                        [10, 64, 56, 56]          73,984
│    └─Identity_Block: 2-3                         [10, 64, 56, 56]          --
│    │    └─Sequential: 3-3                        [10, 64, 56, 56]          73,984
├─Sequential: 1-2                                  [10, 128, 28, 28]         --
│    └─projection_Block: 2-4                       [10, 128, 28, 28]         --
│    │    └─Sequential: 3-4                        [10, 128, 28, 28]         221,696
│    └─Identity_Bl